# 2.3) Supervised Segmentation
A U-Net is trained here to perform a semantic segmentation of laser points in the LASTEN data set.

# Import statements
Following packages are necessary:

In [1]:
import random
import utils

import tensorflow as tf
import numpy as np
import segmentation_models as sm

from unet import UNet
from unet import preprocess_input as pre_une
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score
from lastengen import LASTENSequence
from matplotlib import pyplot as plt

Segmentation Models: using `tf.keras` framework.


# Checks
The version of tensorflow as well as the GPU support are checked.

In [2]:
print(tf.__version__)
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.list_physical_devices('GPU')

2.0.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Data
Import training images

In [3]:
path_train = r'D:\LASTEN\train'
path_validation = r'D:\LASTEN\validation'
path_test = r'D:\LASTEN\test'

#path_train = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/train'
#path_validation = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/validation'
#path_test = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/test'

width = 768 #384
height = 768 #384

# Seeding
Seeds are set to ensure reproducible training.

In [4]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [5]:
model = UNet(filters=32, input_shape=(height, width, 1))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 768, 768, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 768, 768, 48) 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 768, 768, 48) 192         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 768, 768, 48) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Data Preparation
The generator sequence for training is built.

In [6]:
image_ids = None

train_gen = LASTENSequence(path_train,
                           batch_size=2,
                           image_ids=image_ids,
                           preprocess_input=pre_une,
                           shuffle=True,
                           augment=True,
                           width=width,
                           height=height)

validation_gen = LASTENSequence(path_validation,
                                batch_size=2,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

#test_gen = LASTENSequence(path_train,
#                          batch_size=8,
#                          preprocess_input=pre_une,
#                          width=width,
#                          height=height)

# Training Preparation
Prepare the settings for training the model.

In [7]:
save_path = "experiments/2_3_segmentation"

logger = CSVLogger(save_path + "/result.log")
checker = ModelCheckpoint(save_path + "/weights.{epoch:02d}.hdf5",
                          period=10)

callbacks = [logger, checker]

model.compile(optimizer='adam',
              loss = dice_loss,
              metrics = ['accuracy', iou_score])

# Training
The model is fitted with a generator.

In [8]:
model.fit_generator(train_gen,
                    epochs=300,
                    callbacks=callbacks,
                    validation_data=validation_gen,
                    validation_freq=1,
                    use_multiprocessing=True,
                    workers=10,
                    max_queue_size=32)

Epoch 1/300
21/84 [======>.......................] - ETA: 1:25 - loss: 0.8443 - accuracy: 0.8726 - iou_score: 0.0853

KeyboardInterrupt: 

# Evaluation

In [ ]:
X, y = validation_gen[0]
y_pred = model.predict(X)

plt.imsave(save_path + "/y.png",y[0,:,:,0], cmap="gray")
plt.imsave(save_path + "/y_pred.png",y_pred[0,:,:,0], cmap="gray")

# 2.4) Supervised Segmentation
A U-Net is trained here to perform a semantic segmentation of laser points in the LASTEN data set.

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [ ]:
model = UNet(filters=48, input_shape=(height, width, 1))
model.summary()

# Data Preparation
The generator sequence for training is built.

In [ ]:
image_ids = None

train_gen = LASTENSequence(path_train,
                           batch_size=2,
                           image_ids=image_ids,
                           preprocess_input=pre_une,
                           shuffle=True,
                           augment=True,
                           width=width,
                           height=height)

validation_gen = LASTENSequence(path_validation,
                                batch_size=2,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

#test_gen = LASTENSequence(path_train,
#                          batch_size=8,
#                          preprocess_input=pre_une,
#                          width=width,
#                          height=height)

# Training Preparation
Prepare the settings for training the model.

In [ ]:
save_path = "experiments/2_4_segmentation"

logger = CSVLogger(save_path + "/result.log")
checker = ModelCheckpoint(save_path + "/weights.{epoch:02d}.hdf5",
                          period=10)

callbacks = [logger, checker]

model.compile(optimizer='adam',
              loss = dice_loss,
              metrics = ['accuracy', iou_score])

# Training
The model is fitted with a generator.

In [ ]:
model.fit_generator(train_gen,
                    epochs=300,
                    callbacks=callbacks,
                    validation_data=validation_gen,
                    validation_freq=1,
                    use_multiprocessing=True,
                    workers=10,
                    max_queue_size=32)

# Evaluation

In [16]:
X, y = validation_gen[0]
y_pred = model.predict(X)

plt.imsave(save_path + "/y.png",y[0,:,:,0], cmap="gray")
plt.imsave(save_path + "/y_pred.png",y_pred[0,:,:,0], cmap="gray")